In [ ]:
# Laborator 5: Item-Based Collaborative Filtering
Implementare locală a algoritmului folosind Pandas și Scikit-Learn.

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Setare seed pentru rezultate reproductibile
np.random.seed(42)

In [3]:
# Citim datele
books_df = pd.read_csv('best-selling-books.csv')
users_df = pd.read_csv('people.csv')

# Generăm ID-uri pentru cărți (la fel ca în Lab 1)
books_df['item_id'] = range(1, len(books_df) + 1)
books_df['item_id'] = books_df['item_id'].astype(str)

# Mapăm ID-urile userilor
users_df['user_id'] = users_df['SP ID'].astype(str)

print(f"Cărți încărcate: {len(books_df)}")
print(f"Useri încărcați: {len(users_df)}")

Cărți încărcate: 174
Useri încărcați: 33


In [4]:
interactions_list = []

for user_id in users_df['user_id']:
    # Fiecare user a citit între 3 și 10 cărți random
    num_books = np.random.randint(3, 11)
    random_books = np.random.choice(books_df['item_id'], num_books, replace=False)
    
    for book_id in random_books:
        # Generăm un rating între 1 și 5
        rating = np.random.randint(1, 6)
        interactions_list.append({'user_id': user_id, 'item_id': book_id, 'rating': rating})

interactions_df = pd.DataFrame(interactions_list)
print(f"Interacțiuni generate: {len(interactions_df)}")
interactions_df.head()

Interacțiuni generate: 206


,user_id,item_id,rating
0,SP01,157,1
1,SP01,146,4
2,SP01,102,5
3,SP01,128,1
4,SP01,142,3


In [5]:
# 1. Pivot Table: Rânduri = Useri, Coloane = Cărți
user_item_matrix = interactions_df.pivot_table(index='user_id', columns='item_id', values='rating').fillna(0)

# 2. Similaritate între ITEMI (transpunem matricea)
# Rezultă o matrice simetrică (Item x Item)
item_similarity = cosine_similarity(user_item_matrix.T)

# Transformăm în DataFrame pentru a păstra ID-urile
item_sim_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

print("Dimensiune matrice similaritate:", item_sim_df.shape)

Dimensiune matrice similaritate: (119, 119)


In [6]:
def get_recommendations(user_id, k=5):
    if user_id not in user_item_matrix.index:
        return "User necunoscut"
    
    # Rating-urile date de userul țintă
    user_ratings = user_item_matrix.loc[user_id]
    known_items = user_ratings[user_ratings > 0].index
    
    scores = {}
    
    # Iterăm prin toate cărțile posibile
    for item in item_sim_df.index:
        if item in known_items:
            continue # Nu recomandăm ce a citit deja
            
        numerator = 0
        denominator = 0
        
        # Calculăm scorul bazat pe similaritatea cu cărțile deja citite
        for known_item in known_items:
            sim = item_sim_df.loc[item, known_item]
            r = user_ratings[known_item]
            
            numerator += sim * r
            denominator += sim
            
        if denominator > 0:
            scores[item] = numerator / denominator
        else:
            scores[item] = 0
            
    # Sortăm descrescător
    sorted_items = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:k]
    
    # Formăm rezultatul
    results = []
    for item_id, score in sorted_items:
        title = books_df[books_df['item_id'] == item_id]['Book'].values[0]
        results.append((title, round(score, 2)))
        
    return results

In [7]:
# Testăm pentru un user (de exemplu SP01)
test_user = 'SP01'
recs = get_recommendations(test_user)

print(f"Recomandări pentru {test_user}:")
for title, score in recs:
    print(f"Carre: {title} | Scor estimat: {score}")

Recomandări pentru SP01:
Carre: Dune | Scor estimat: 5.0
Carre: Where the Crawdads Sing | Scor estimat: 5.0
Carre: Becoming | Scor estimat: 5.0
Carre: Ronia, the Robber's Daughter | Scor estimat: 5.0
Carre: The Cat in the Hat | Scor estimat: 5.0
